# Pràctica 7 — Mesura de **G**: ajust no lineal de \(P(t)\) (Binder)

Aquest notebook fa un ajust no lineal del senyal \(P(t)\) amb el model d’oscil·lador amortit (solució del guió) i mostra:
- dades amb barres d’error (`sx`, `sy`),
- corba ajustada,
- banda d’incertesa (~68%) a partir de la covariància (si es pot calcular),
- residus,
- export automàtic dels resultats (CSV/TXT + ZIP) i enllaços per descarregar-los.

## Important (Binder)
- En Binder, la sessió és temporal. Després d’exportar, descarrega el **ZIP**.
- Recomanat obrir amb Notebook clàssic (widgets):
  - `...?urlpath=notebooks/<ruta_al_notebook>.ipynb`


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

%matplotlib inline

import sys, scipy, numpy as np
print("Versions:")
print(" Python:", sys.version.split()[0])
print(" NumPy :", numpy.__version__)
print(" SciPy :", scipy.__version__)


## 1) Pujar el CSV des del navegador (opcional)

Si ja tens el CSV al mateix directori que el notebook, pots saltar aquesta part.

1. Executa la cel·la.
2. Prem “Puja CSV” i selecciona el teu fitxer.
3. Després executa la cel·la “Carrega dades”.


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Fitxer per defecte
CSV_PATH = "mesuresG.csv"

csv_path_box = widgets.Text(
    value=CSV_PATH,
    description="CSV_PATH",
    disabled=False,
    layout=widgets.Layout(width="550px")
)

uploader = widgets.FileUpload(
    accept=".csv",
    multiple=False,
    description="Puja CSV"
)

out_upload = widgets.Output()
display(widgets.VBox([csv_path_box, uploader, out_upload]))

def _first_uploaded_file(value):
    # Compatibilitat ipywidgets 7 i 8
    if value is None:
        return None, None
    if isinstance(value, dict):
        if len(value) == 0:
            return None, None
        name, info = next(iter(value.items()))
        content = info.get("content", None) if isinstance(info, dict) else getattr(info, "content", None)
        return name, content
    if isinstance(value, (tuple, list)):
        if len(value) == 0:
            return None, None
        f0 = value[0]
        if isinstance(f0, dict):
            return f0.get("name", None), f0.get("content", None)
        return getattr(f0, "name", None), getattr(f0, "content", None)
    return None, None

def _to_bytes(content):
    if content is None:
        return None
    if hasattr(content, "tobytes"):
        return content.tobytes()
    try:
        return bytes(content)
    except Exception:
        return None

def _on_upload_change(change):
    global CSV_PATH
    with out_upload:
        out_upload.clear_output(wait=True)
        name, content = _first_uploaded_file(uploader.value)
        if name is None or content is None:
            print("No s'ha detectat cap fitxer. Torna-ho a provar.")
            return
        data = _to_bytes(content)
        if data is None:
            print("No he pogut llegir el contingut del fitxer.")
            return
        with open(name, "wb") as f:
            f.write(data)
        CSV_PATH = name
        csv_path_box.value = name
        print(f"Fitxer pujat: {name}")
        print("Ara executa la cel·la: 'Carrega dades'")

uploader.observe(_on_upload_change, names="value")


## 2) Carrega dades

El CSV ha de tindre columnes: `x, y, sy, sx`

- `x`: temps (s)
- `y`: posició \(P\) (unitats del regle)
- `sy`: error en `y`
- `sx`: error en `x`


In [ ]:
# ------------------------------------
# Carrega dades
# ------------------------------------
df = pd.read_csv(CSV_PATH)

required = {"x", "y", "sy", "sx"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Falten columnes al CSV: {missing}. Necessites: {sorted(required)}")

x  = df["x"].to_numpy(dtype=float)
y  = df["y"].to_numpy(dtype=float)
sy = df["sy"].to_numpy(dtype=float)
sx = df["sx"].to_numpy(dtype=float)

# Ordena per temps
idx = np.argsort(x)
x, y, sy, sx = x[idx], y[idx], sy[idx], sx[idx]

df.head()


In [ ]:
# Vista ràpida de les dades
plt.figure(figsize=(8.5, 4.6))
plt.errorbar(x, y, yerr=sy, xerr=sx, fmt="o", capsize=2)
plt.xlabel("t (s)")
plt.ylabel("P (unitats del regle)")
plt.grid(True)
plt.tight_layout()
plt.show()


## 3) Model de l’ajust

Fem servir:

$$
P(t)=P_e + S_l\Big[1 - e^{-\alpha \Delta t/2}\Big(\cos(\omega_1\Delta t)+\frac{\alpha}{2\omega_1}\sin(\omega_1\Delta t)\Big)\Big],
\qquad \omega_1=\frac{2\pi}{T},
\qquad \Delta t = t - t_\mathrm{shift}.
$$

Paràmetres: `Pe, Sl, alpha, T, t_shift`.

### Errors en X i Y
`curve_fit` no tracta directament `sx`. Ací usem una aproximació habitual:

$$
\sigma_\mathrm{eff}^2 = \sigma_y^2 + \left(\frac{dP}{dt}\sigma_x\right)^2,
$$

i iterem el fit unes quantes vegades.


In [ ]:
# ------------------------------------
# Model
# ------------------------------------
def model_func(t, Pe, Sl, alpha, T, t_shift):
    dt = t - t_shift
    w1 = 2*np.pi / T
    expo = np.exp(-alpha*dt/2.0)
    return Pe + Sl*(1.0 - expo*(np.cos(w1*dt) + (alpha/(2*w1))*np.sin(w1*dt)))

PARAM_NAMES = ["Pe", "Sl", "alpha", "T", "t_shift"]


In [ ]:
# ------------------------------------
# Seeds (valors inicials) automàtics
# ------------------------------------
def estimate_seeds(x, y):
    Pe0 = float(np.median(y[:min(5, len(y))]))
    Sl0 = float(np.median(y[-min(5, len(y)) :]) - Pe0)

    peaks = []
    for i in range(1, len(y)-1):
        if (y[i] > y[i-1]) and (y[i] >= y[i+1]):
            peaks.append(i)

    if len(peaks) >= 2:
        T0 = float(np.median(np.diff(x[peaks])))
    else:
        T0 = float((x.max()-x.min())/4.0) if len(x) > 1 else 500.0

    alpha0 = 4e-4
    t_shift0 = float(x.min())
    return [Pe0, Sl0, alpha0, T0, t_shift0]

P0 = estimate_seeds(x, y)
print("P0 =", P0)


In [ ]:
# ------------------------------------
# Ajust no lineal amb sigma_eff
# ------------------------------------
ymin, ymax = float(y.min()), float(y.max())
rng = (ymax - ymin) if ymax > ymin else 1.0

BOUNDS = (
    [ymin - 5*rng,  -10*rng, 0.0,   10.0,   float(x.min() - 2000.0)],
    [ymax + 5*rng,   10*rng, 0.05, 2000.0,  float(x.min() + 2000.0)]
)

def dPdt_numeric(t, params, eps=1e-2):
    return (model_func(t+eps, *params) - model_func(t-eps, *params)) / (2*eps)

def fit_with_sigma_eff(x, y, sy, sx, p0, bounds, n_iter=5):
    sigma = sy.copy().astype(float)
    popt = None
    pcov = None

    for _ in range(int(n_iter)):
        popt, pcov = curve_fit(
            model_func, x, y,
            p0=(p0 if popt is None else popt),
            sigma=sigma, absolute_sigma=True,
            bounds=bounds,
            maxfev=250000
        )
        dp = dPdt_numeric(x, popt, eps=1e-2)
        sigma = np.sqrt(sy**2 + (dp*sx)**2)

    return popt, pcov, sigma

popt, pcov, sigma_eff = fit_with_sigma_eff(x, y, sy, sx, P0, BOUNDS, n_iter=5)
perr = np.sqrt(np.diag(pcov))

resid = y - model_func(x, *popt)
chi2 = float(np.sum((resid / sigma_eff)**2))
dof = int(len(x) - len(popt))
chi2_red = chi2 / max(dof, 1)

print("Resultats del fit")
print("chi2     =", chi2)
print("chi2_red =", chi2_red)
print("dof      =", dof)
print("")
for name, val, err in zip(PARAM_NAMES, popt, perr):
    print(f"{name:8s} = {val: .6g}  ± {err:.2g}")


## 4) Banda d’incertesa (~68%) a partir de la covariància

Mostregem paràmetres d’una normal multivariant amb mitjana `popt` i covariància `pcov`,
aplicant restriccions simples (alpha ≥ 0, T > 0 i bounds del fit).


In [ ]:
# ------------------------------------
# Banda d'incertesa (opcional)
# ------------------------------------
def make_psd(cov, jitter=1e-15):
    cov = 0.5*(cov + cov.T)
    w, V = np.linalg.eigh(cov)
    w = np.clip(w, 0.0, None)
    cov_psd = (V*w) @ V.T
    scale = float(np.max(np.diag(cov_psd))) if cov_psd.size else 1.0
    cov_psd = cov_psd + (jitter*(scale if scale > 0 else 1.0))*np.eye(cov_psd.shape[0])
    return cov_psd

def mc_band_constrained(x_grid, popt, pcov, bounds, n_draws=1200, seed=12345):
    rng = np.random.default_rng(seed)
    cov_psd = make_psd(np.array(pcov, dtype=float))

    lo_b = np.array(bounds[0], dtype=float)
    hi_b = np.array(bounds[1], dtype=float)

    accepted = []
    target = int(n_draws)
    batch = max(500, target)
    attempts = 0

    while len(accepted) < target and attempts < 50:
        draws = rng.multivariate_normal(mean=np.array(popt, float), cov=cov_psd, size=batch)
        attempts += 1

        alpha = draws[:, 2]
        T = draws[:, 3]
        ok = (alpha >= 0.0) & (T > 0.0)
        ok = ok & np.all(draws >= lo_b, axis=1) & np.all(draws <= hi_b, axis=1)

        draws = draws[ok]
        if draws.size == 0:
            continue

        for p in draws:
            yv = model_func(x_grid, *p)
            if np.all(np.isfinite(yv)):
                accepted.append(yv)
            if len(accepted) >= target:
                break

    if len(accepted) == 0:
        return None, None

    ys = np.array(accepted[:target])
    lo = np.percentile(ys, 16, axis=0)
    hi = np.percentile(ys, 84, axis=0)
    return lo, hi

x_grid = np.linspace(float(x.min()), float(x.max()), 2500)
y_fit  = model_func(x_grid, *popt)

lo, hi = mc_band_constrained(x_grid, popt, pcov, BOUNDS, n_draws=1200, seed=12345)


In [ ]:
# Plot principal: dades + fit + banda (si existeix)
plt.figure(figsize=(9.2, 5.0))
plt.errorbar(x, y, yerr=sy, xerr=sx, fmt="o", capsize=2, label="Dades")
plt.plot(x_grid, y_fit, linewidth=2, label="Model ajustat")
if lo is not None and hi is not None:
    plt.fill_between(x_grid, lo, hi, alpha=0.25, label="Banda ~68% (paràmetres)")
plt.xlabel("t (s)")
plt.ylabel("P (unitats del regle)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Residus
plt.figure(figsize=(9.2, 3.3))
plt.axhline(0, linewidth=1)
plt.errorbar(x, y - model_func(x, *popt), yerr=sy, xerr=sx, fmt="o", capsize=2)
plt.xlabel("t (s)")
plt.ylabel("residu (P - model)")
plt.grid(True)
plt.tight_layout()
plt.show()


## 5) Export i descàrrega (Binder)

Aquesta cel·la:
- crea una carpeta `resultats_fitG/`,
- guarda les figures PNG,
- exporta CSV/TXT i un ZIP amb tot.

Si els enllaços no descarreguen:
- ves a **File → Open**, entra a `resultats_fitG/` i baixa el ZIP.


In [ ]:
import csv
import zipfile
from pathlib import Path
import urllib.parse as up
from IPython.display import HTML, display

OUTDIR = Path("resultats_fitG")
PREFIX = "fitG"
OUTDIR.mkdir(parents=True, exist_ok=True)

# Guarda figures (les 2 últimes figures actives no són sempre les que volem; recreem i guardem explícitament)
# Figura fit
plt.figure(figsize=(9.2, 5.0))
plt.errorbar(x, y, yerr=sy, xerr=sx, fmt="o", capsize=2, label="Dades")
plt.plot(x_grid, y_fit, linewidth=2, label="Model ajustat")
if lo is not None and hi is not None:
    plt.fill_between(x_grid, lo, hi, alpha=0.25, label="Banda ~68% (paràmetres)")
plt.xlabel("t (s)")
plt.ylabel("P (unitats del regle)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(OUTDIR / f"{PREFIX}_fit.png", dpi=200)
plt.close()

# Figura residus
plt.figure(figsize=(9.2, 3.3))
plt.axhline(0, linewidth=1)
plt.errorbar(x, y - model_func(x, *popt), yerr=sy, xerr=sx, fmt="o", capsize=2)
plt.xlabel("t (s)")
plt.ylabel("residu (P - model)")
plt.grid(True)
plt.tight_layout()
plt.savefig(OUTDIR / f"{PREFIX}_residus.png", dpi=200)
plt.close()

files = []

# TXT resum
txt_path = OUTDIR / f"{PREFIX}_resultats_fit.txt"
with open(txt_path, "w", encoding="utf-8") as f:
    f.write("Resultats del fit (Pràctica 7 — Mesura de G)\n")
    f.write(f"chi2: {chi2}\n")
    f.write(f"chi2_red: {chi2_red}\n")
    f.write(f"dof: {dof}\n\n")
    f.write("Paràmetres (valor ± 1σ)\n")
    for name, val, err in zip(PARAM_NAMES, popt, perr):
        f.write(f"{name}: {val} +/- {err}\n")
files.append(txt_path)

# CSV paràmetres
par_csv = OUTDIR / f"{PREFIX}_resultats_fit.csv"
with open(par_csv, "w", newline="", encoding="utf-8") as f:
    wri = csv.writer(f)
    wri.writerow(["parametre", "valor", "error_1sigma"])
    for name, val, err in zip(PARAM_NAMES, popt, perr):
        wri.writerow([name, float(val), float(err)])
    wri.writerow([])
    wri.writerow(["chi2", float(chi2)])
    wri.writerow(["chi2_red", float(chi2_red)])
    wri.writerow(["dof", int(dof)])
files.append(par_csv)

# Residus CSV
resid_csv = OUTDIR / f"{PREFIX}_residus.csv"
y_model = model_func(x, *popt)
with open(resid_csv, "w", newline="", encoding="utf-8") as f:
    wri = csv.writer(f)
    wri.writerow(["t", "P", "model", "residu", "sy", "sx", "sigma_eff"])
    for i in range(len(x)):
        wri.writerow([float(x[i]), float(y[i]), float(y_model[i]), float(y[i]-y_model[i]),
                      float(sy[i]), float(sx[i]), float(sigma_eff[i])])
files.append(resid_csv)

# Corba + banda CSV
curve_csv = OUTDIR / f"{PREFIX}_corba_fit.csv"
with open(curve_csv, "w", newline="", encoding="utf-8") as f:
    wri = csv.writer(f)
    if lo is None or hi is None:
        wri.writerow(["t_grid", "P_fit"])
        for t_, p_ in zip(x_grid, y_fit):
            wri.writerow([float(t_), float(p_)])
    else:
        wri.writerow(["t_grid", "P_fit", "P_lo_68", "P_hi_68"])
        for t_, p_, l_, h_ in zip(x_grid, y_fit, lo, hi):
            wri.writerow([float(t_), float(p_), float(l_), float(h_)])
files.append(curve_csv)

# Covariància
cov_csv = OUTDIR / f"{PREFIX}_covariancia.csv"
with open(cov_csv, "w", newline="", encoding="utf-8") as f:
    wri = csv.writer(f)
    wri.writerow([""] + PARAM_NAMES)
    pc = np.array(pcov, dtype=float)
    for i, row in enumerate(pc):
        wri.writerow([PARAM_NAMES[i]] + [float(v) for v in row])
files.append(cov_csv)

# ZIP amb tot
zip_path = OUTDIR / f"{PREFIX}_resultats.zip"
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for fp in files + [OUTDIR / f"{PREFIX}_fit.png", OUTDIR / f"{PREFIX}_residus.png"]:
        zf.write(fp, arcname=fp.name)
files.append(zip_path)

print("Creats:")
for fp in files:
    print(" -", fp)

# Enllaços de descàrrega (més robustos)
print("\nEnllaços de descàrrega:")
for fp in files:
    rel = fp.as_posix()
    href = "files/" + up.quote(rel)
    display(HTML(f'<a href="{href}" download="{fp.name}">Descarregar {fp.name}</a>'))
